# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Packages

In [1]:
from __future__ import division
import sklearn
import pandas as pd 
import numpy as np 
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv("amazon_baby.csv", encoding="utf8")

Now, let us see a preview of what the dataset looks like.

In [3]:
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
# products[269:270]
products.loc[[269]]

,name,review,rating
269,The First Years Massaging Action Teether,A favorite in our house!,5


**IMPORTANT**. Make sure to fill n/a values in the review column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the review columns as follows:

In [5]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. You can research more about a smart handling of punctuations. **Keyword: Tokenization**

In [6]:
def remove_punctuation(text):
    import string
    return str(text).translate(str.maketrans('','',string.punctuation)) 

products['review_clean'] = products['review'].apply(remove_punctuation)

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1
...,...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5,Such a great idea very handy to have and look ...,1
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5,This product rocks It is a great blend of fun...,1
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5,This item looks great and cool for my kidsI kn...,1
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5,I am extremely happy with this product I have ...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `random_state=42` so that everyone gets the same result.

In [9]:
from sklearn.model_selection import train_test_split
X = products['review_clean'].tolist()
y = products['sentiment'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(len(X_train))
print(len(X_test))

111723
55029


# Build the word count vector for each review

6. We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.

Compute the occurrences of the words in each review and collect them into a row vector.

Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.

Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix

train_matrix = vectorizer.fit_transform(X_train)
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(X_test)

Keep in mind that the test data must be transformed in the same way as the training data.



# Train a sentiment classifier with logistic regression

7. Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

**Note:** This line may take 1-2 minutes.

In [127]:
# Use sklearn's LogisticRegression
# Your code here
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(solver='lbfgs', max_iter=1000,
                          random_state=1356, warm_start=True)
sentiment_model.fit(train_matrix, y_train)
weights = sentiment_model.coef_

In [128]:
weights

array([[-3.42518174e-01,  4.77943498e-05,  2.31015990e-02, ...,
         7.98648514e-03,  1.34258507e-05, -3.37307791e-05]])

8. There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

**Quiz Question:** How many weights are >= 0?

In [129]:
# Your code here 
print(np.sum(weights >= 0))

80243


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **X_sample_test_data**.

In [14]:
X_sample_test_data = X_test[10:13]
y_sample_test_data = y_test[10:13]
print(X_sample_test_data)
print(y_sample_test_data)

['I like the idea but the problemsawkward closing zipper requires two hands but you are likely to have the cups of the pump fall out or slip around while zippingsize range XSmedium got to be kidding its very picky trying to get the width of the back and front so that your nipples fall right in the center of the front nipple openings and if theyre off to the side the cups of the pump slide and nipples rub the insides during pumping OUCHnot worth the pricenot a bra Id want to wear all day and getting it on and off also not convenient', 'This is my 2nd Moby just wanted another color to switch off with  Thought it would be nice to keep one in the house and one in the car However just had to pass one of mine on to a friend who tried my moby out and loved it so much  I guess Ill have to get yet another  Moby wraps are excellent for your back and shoulders and the child loves being close to mommy or whomever is wearing the wrap  I even had my almost 3 year old daughter in it a couple weeks ag

Let's dig deeper into the second row of the **X_sample_test_data**. Here's the full review:

In [15]:
X_sample_test_data[1]

'This is my 2nd Moby just wanted another color to switch off with  Thought it would be nice to keep one in the house and one in the car However just had to pass one of mine on to a friend who tried my moby out and loved it so much  I guess Ill have to get yet another  Moby wraps are excellent for your back and shoulders and the child loves being close to mommy or whomever is wearing the wrap  I even had my almost 3 year old daughter in it a couple weeks ago although I use it most with my newborn and will continue to as she grows  This is my 3rd baby using Mobys with I love them  They are so comfortable and are simple to put on after following the instructions and doing it a few times  I wear it going out sometimes so its convenient to just slip the baby in when I arrive at my destination  Breathable cotton is great and it has a little bit of a stretch to it  Its also nice to be able to face the baby outward once they can hold their heads up really well  Mine have always loved to face o

That review seems pretty positive.

Now, let's see what the first row of the **X_sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [16]:
X_sample_test_data[0]

'I like the idea but the problemsawkward closing zipper requires two hands but you are likely to have the cups of the pump fall out or slip around while zippingsize range XSmedium got to be kidding its very picky trying to get the width of the back and front so that your nipples fall right in the center of the front nipple openings and if theyre off to the side the cups of the pump slide and nipples rub the insides during pumping OUCHnot worth the pricenot a bra Id want to wear all day and getting it on and off also not convenient'

We will now make a **class** prediction for the **X_sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  
We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in sample_test_data. In scikit-learn, you can call the decision_function() function.

**Hint**: You'd probably need to convert sample_test_data into the sparse matrix format first.

In [130]:
sample_test_matrix = vectorizer.transform(X_sample_test_data)
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[-1.33018937 16.12953911  5.96413286]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [136]:
apply_threshold = np.vectorize(lambda x: 1. if x > 0  else -1.)
y = apply_threshold(scores)
print(y)


[-1.  1.  1.]


**Checkpoint**: Make sure your class predictions match with the ones obtained from sentiment_model. The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

In [137]:
print("Class predictions according to GraphLab Create:")
print(sentiment_model.predict(sample_test_matrix))

Class predictions according to GraphLab Create:
[-1  1  1]


### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [139]:
apply = np.vectorize(lambda x: 1/(1 + math.exp(-1 * x)))
Prediction = apply(scores)
Prediction

array([0.20912804, 0.9999999 , 0.99743731])

**Checkpoint**: Make sure your probability predictions match the ones obtained from sentiment_model.

In [140]:
print(sentiment_model.predict_proba(sample_test_matrix))

[[7.90871957e-01 2.09128043e-01]
 [9.88621667e-08 9.99999901e-01]
 [2.56268732e-03 9.97437313e-01]]


**Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

13. We now turn to examining the full test dataset, test_data, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the sentiment_model, find the 20 reviews in the entire test_data with the highest probability of being classified as a positive review. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:

1. Make probability predictions on test_data using the sentiment_model.
2. Sort the data according to those predictions and pick the top 20.

In [142]:
X_test_matrix = vectorizer.transform(X_test)
scores_X = sentiment_model.predict_proba(X_test_matrix)

(55029, 2)

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [124]:
top_20_pos = np.argsort(-scores_X[:,1])[:20]
top_20_neg = np.argsort(-scores_X[:,0])[:20]
print(top_20_pos)
print(top_20_neg)
print(scores_X[top_20_pos])
print(scores_X[top_20_neg])

[29191  7040 24279 46653 29592 31353 40155 31164 11037 49377 36188 38872
  4938  8508 38620  4727 36738 52684 36018 51296]
[30933  4809   781 12532 40917 53758 25937  6190 12545 54944 33413  7003
 32579 39123 42653  1591 22565  3719 41376  2657]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1.00000000e+00 4.62278094e-22]
 [1.00000000e+00 2.90291833e-19]
 [1.00000000e+00 7.03536772e-17]
 [1.00000000e+00 1.10910229e-15]
 [1.00000000e+00 2.08546747e-15]
 [1.00000000e+00 7.95238968e-15]
 [1.00000000e+00 1.10608008e-14]
 [1.00000000e+00 7.24174500e-14]
 [1.00000000e+00 3.14666722e-13]
 [1.00000000e+00 3.74951486e-13]
 [1.00000000e+00 5.23370912e-13]
 [1.00000000e+00 2.73832400e-12]
 [1.00000000e+00 2.82515583e-12]
 [1.00000000e+00 3.96366993e-12]
 [1.00000000e+00 5.56062853e-12]
 [1.00000000e+00 1.06683523e-11]
 [1.00000000e+00 1.21369984e-11]
 [1.00000000

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [57]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    total_examples = np.shape(true_labels)
    prediction = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct_classified = np.sum(prediction==true_labels)
    
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct_classified / total_examples
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [58]:
get_classification_accuracy(logr, X_test_matrix, y_test)

array([0.9311454])

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [59]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [60]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [161]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(X_train)
test_matrix_word_subset = vectorizer_word_subset.transform(X_test)

## Train a logistic regression model on a subset of data

Now build a logistic regression classifier with train_matrix_word_subset as features and sentiment as the target. Call this model **simple_model**.

In [162]:
simple_model = LogisticRegression(solver='lbfgs', max_iter=1000,
                                  random_state=1356, warm_start=True)
simple_model.fit(train_matrix_word_subset, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1356, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=True)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [163]:
get_classification_accuracy(simple_model, test_matrix_word_subset, y_test)

array([0.86750622])

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [148]:
print(np.sum(simple_model.coef_ >= 0))

10


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [167]:
index = np.argsort(-simple_model.coef_[0])
print(simple_model.coef_[0][index])


[ 1.70332306  1.51336294  1.35847482  1.17962337  0.9123805   0.51902236
  0.49598179  0.20621029  0.08451118  0.06100797 -0.20315524 -0.30754791
 -0.35082121 -0.49910675 -0.62132672 -0.94032203 -1.64291976 -1.99816979
 -2.08781357 -2.38775837]


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [115]:
# Your code here 

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [156]:
acc_sentiment = get_classification_accuracy(sentiment_model,
                                            train_matrix,
                                            y_train)
print(acc_sentiment)

[0.97082964]


Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [164]:
acc_simple = get_classification_accuracy(simple_model,
                                         train_matrix_word_subset,
                                         y_train)
print(acc_simple)

[0.86723414]


**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [153]:
acc_sentiment_test = get_classification_accuracy(sentiment_model,
                                                 test_matrix,
                                                 y_test)
print(acc_sentiment_test)

[0.9311454]


Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [165]:
acc_simple_test = get_classification_accuracy(simple_model,
                                              test_matrix_word_subset,
                                              y_test)
print(acc_simple_test)

[0.86750622]


**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [120]:
num_positive  = np.sum(np.array(y_train) == 1)
num_negative = np.sum(np.array(y_train) == -1)
print(num_positive)
print(num_negative)

94005
17718


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [160]:
num_positive  = np.sum(np.array(y_test) == 1)
num_negative = np.sum(np.array(y_test) == -1)
print(np.round(num_positive/np.shape(y_test)[0],2))

0.84


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

In [122]:
# Your code here 